In [1]:
##########################################################################
# Created on Sat Nov 11 23:20:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 10.4.1.Day convention

In [2]:
import datetime as dt

def accrued_interest(par,c,m,t1,t2,t3,t4,rule):
    '''Define a function to calculate accrued interest based on different day convention
    par:par of bond
    c:coupon
    m:coupon payment frequency per year
    t1:beginning of non reference period
    t2:ending of non reference period
    t3:beginning of reference period
    t4:ending of reference period
    rule:date convention. 'actual/actual','actual/360',otherwise 'actual/365' '''
    d1=(t2-t1).days
    if rule=='actual/actual':
        d2=(t4-t3).days
        interest=(d1/d2)*par*c/m
    elif rule=='actual/360':
        interest=(d1/360)*par*c
    else:
        interest=(d1/365)*par*c
    return interest

In [3]:
par_TB06=1e6
C_TB06=0.0268
m_TB06=2

t1_TB06=dt.datetime(2020,5,28)
t2_TB06=dt.datetime(2020,10,16)
t3_TB06=dt.datetime(2020,5,21)
t4_TB06=dt.datetime(2020,11,21)

R1_TB06=accrued_interest(par=par_TB06,c=C_TB06,m=m_TB06,t1=t1_TB06,t2=t2_TB06,t3=t3_TB06,t4=t4_TB06,rule='actual/actual')
R2_TB06=accrued_interest(par=par_TB06,c=C_TB06,m=m_TB06,t1=t1_TB06,t2=t2_TB06,t3=t3_TB06,t4=t4_TB06,rule='actual/360')
R3_TB06=accrued_interest(par=par_TB06,c=C_TB06,m=m_TB06,t1=t1_TB06,t2=t2_TB06,t3=t3_TB06,t4=t4_TB06,rule='actual/365')

print('accrued based on actual/actual day convention is ',round(R1_TB06,2))
print('accrued based on actual/360 day convention is ',round(R2_TB06,2))
print('accrued based on actual/365 day convention is ',round(R3_TB06,2))

accrued based on actual/actual day convention is  10268.48
accrued based on actual/360 day convention is  10496.67
accrued based on actual/365 day convention is  10352.88


# 10.4.2.Quote of Treasury bond

In [4]:
import numpy as np

t_begin=dt.datetime(2020,5,21)
t_mature=dt.datetime(2030,5,21)
t_pricing=dt.datetime(2020,8,6)
t_next1=dt.datetime(2020,11,21)

N=((t_mature-t_pricing).days//365+1)*m_TB06#number of coupon payment times
tenor=(t_next1-t_pricing).days/365 #time(year) between pricing date and next coupon payment date
t_list=np.arange(N)/2+tenor #time(year) between pricing date and every coupon payment date
t_list

array([0.29315068, 0.79315068, 1.29315068, 1.79315068, 2.29315068,
       2.79315068, 3.29315068, 3.79315068, 4.29315068, 4.79315068,
       5.29315068, 5.79315068, 6.29315068, 6.79315068, 7.29315068,
       7.79315068, 8.29315068, 8.79315068, 9.29315068, 9.79315068])

In [5]:
import utils

bond_par=100
y_TB06=0.0301 #YTM

dirty_price=utils.Bondprice_onediscount(C=C_TB06,M=bond_par,m=m_TB06,y=y_TB06,t=t_list)
print('Dirty price of TB06 is ',round(dirty_price,4))

Dirty price of TB06 is  97.5823


In [7]:
bond_interest=accrued_interest(par=bond_par,c=C_TB06,m=m_TB06,t1=t_begin,t2=t_pricing,t3=t_begin,t4=t_next1,rule='actual/actual')
print('Accured interest of TB06 is ',round(bond_interest,4))

Accured interest of TB06 is  0.5608


In [8]:
clean_price=dirty_price-bond_interest
print('Clean price of TB06 is ',round(clean_price,4))

Clean price of TB06 is  97.0216


# 10.4.3.Invoice price of Treasury bond futures

$$\mathit{Invoice \ price \ of \ futures=Price \ of \ futures \times Conversion \ factor + Accured \ interest}$$

## 1.Conversion factor

$$CF=\frac{1}{\left(1+\frac{r}{m}\right)^{\frac{xm}{12}}}\left[\frac{c}{m}+\frac{c}{r}+\frac{\left(1-\frac{c}{r}\right)}{\left(1+\frac{r}{m}\right)^{n-1}}\right]-\frac{c}{m}\left(1-\frac{xm}{12}\right)\tag{10-17}$$
where,\
r:coupon of underlying asset, 3% as an example \
x:The number of months from the delivery month of treasury bond futures to the next interest payment month of deliverable bond\
n:Remaining interest payment times of deliverable bond after the delivery date of treasury bond futures\
c:Coupon of deliverable bond\
m:Coupon payment frequency per year

In [9]:
def CF(x,n,c,m):
    '''Define a function to calculate conversion factor of deliverable bond
    the default coupon of underlying asset is 3%
    x:The number of months from the delivery month of treasury bond futures to the next interest payment month of deliverable bond
    n:Remaining interest payment times of deliverable bond after the delivery date of treasury bond futures
    c:Coupon of deliverable bond
    m:Coupon payment frequency per year'''
    A=1/pow(1+0.03/m,x*m/12)
    B=c/m+c/0.03+(1-c/0.03)/pow(1+0.03/m,n-1)
    D=c*(1-x*m/12)/m
    value=A*B-D
    return value

In [10]:
t_settle1=dt.datetime(2020,12,16)
t_next2=dt.datetime(2021,5,21)

months=12+(t_next2.month-t_settle1.month)
N2=((t_mature-t_settle1).days//365)*m_TB06+1

CF_TB06=CF(x=months,n=N2,c=C_TB06,m=m_TB06)
print('conversion factor of TB06 for T2012 contract is ',round(CF_TB06,4))

conversion factor of TB06 for T2012 contract is  0.9739


## 2.Accured interest

$$Accrued \ interest=\frac{coupon \ of \ deliverable \ bond}{coupon \ payment \ frequency \ every \ year}\times
\frac{second \ settlement \ date \ of \ future-Last \ interest \ payment \ date \ of \ deliverable \ bonds}
{Actual \ days \ of \ the \ current \ interest \ payment \ cycle}$$

In [11]:
t_settle2=dt.datetime(2020,12,15)
bond_interest2=accrued_interest(par=bond_par,c=C_TB06,m=m_TB06,t1=t_next1,t2=t_settle2,t3=t_next1,t4=t_next2,rule='actual/actual')
print('accrued interest of TB06 as deliverable bond is ',round(bond_interest2,4))

accrued interest of TB06 as deliverable bond is  0.1777


# 10.4.4.CTD of Treasury bond futures

In [12]:
def CTD_cost(price1,price2,CF,name):
    '''Define a function to calculate delivery cost and find the cheapest delivery bond for settlement
    price1:Clean price of deliverable bonds in array
    price2:Price of Treasury bond future
    CF:Conversion factor of deliverable bonds in array
    name:Name of deliverable bonds in array'''
    import pandas as pd
    
    cost=price1-price2*CF
    cost=pd.DataFrame(data=cost,index=name,columns=['Delivery cost'])
    CTD_bond=cost.idxmin()#Return the index at which the minimum value can be obtained
    #print(CTD_bond)
    CTD_bond=CTD_bond.rename(index={'Delivery cost':'Cheapest deliverable bond'})
    return cost,CTD_bond

In [13]:
price_3bond=np.array([94.9870,98.6951,96.1669])
price_T2012=97.225
CF_3bond=np.array([0.9739,1.0101,0.9884])
name_3bond=np.array(['TB2006','TB1915','TB2004'])

result=CTD_cost(price1=price_3bond,price2=price_T2012,CF=CF_3bond,name=name_3bond)
result[0]

,Delivery cost
TB2006,0.299572
TB1915,0.488128
TB2004,0.069710


In [14]:
result[-1]

Cheapest deliverable bond    TB2004
dtype: object

# 10.4.5.Duration based hedge ratio

$$\Delta P \approx -PD_P\Delta y\tag{10-18}$$
$$\Delta V_F \approx -V_F D_F\Delta y\tag{10-19}$$
where,\
P:Price of bond portfolio\
$D_P$:Macaulay duration of bond portfolio\
$D_F$:Macaulay duration of underlying asset of Treasury bond future\
$V_F$:Price of bond future contract per unit\
$\Delta y$:Parallel shift of yield curve

Combine (10-18) and (10-19)\
$$N^*=\frac{\Delta P}{\Delta V_F}=\frac{PD_P}{V_F D_F}\tag{10-20}$$

In [15]:
def N_TBF(Pf,par,value,Df,Dp):
    '''Define a function to calculate number of future contracts for hedging
    Pf:price of bond future
    par:par of underlying asset of future
    value:price of bond portfolio
    Df:Macaulay duration of underlying asset of Treasury bond future(CTD)
    Dp:Macaulay duration of bond portfolio'''
    value_TBF=Pf*par/100
    N=value*Dp/(value_TBF*Df)
    return N

In [16]:
import utils

C_TB04=0.0286
y_TB04=0.0295
m_TB04=2
par_TB04=100

t_T2009=dt.datetime(2020,9,11)# Maturity of future
t1_TB04=dt.datetime(2021,1,16)# Next coupon payment date of CTD
t2_TB04=dt.datetime(2030,7,16)# Maturity of CTD

N_TB04=((t2_TB04-t_T2009).days//365+1)*m_TB04# Remaining number of payment times after future maturity
tenor=(t1_TB04-t_T2009).days/365# The period from the hedging maturity date to the next interest payment date of CTD

t_list=np.arange(N_TB04)/m_TB04+tenor# The term array between the hedging maturity date and the CTD residual cash flow payment date

D_TB04=utils.Mac_Duration(C=C_TB04,M=par_TB04,m=m_TB04,y=y_TB04,t=t_list)
print('Macaulay duration of CTD when future matures is ',round(D_TB04,4))


Macaulay duration of CTD when future matures is  8.608


In [17]:
par_T2009=1e6 #par of underlying asset of future 
price_T2009=99.51
value_fund=1e9
D_fund=8.68

N_T2009=N_TBF(Pf=price_T2009,par=par_T2009,value=value_fund,Df=D_TB04,Dp=D_fund)
print('Number of future contracts used for hedging is ',round(N_T2009,2))

Number of future contracts used for hedging is  1013.33
